# Export DTA File

Generic file export notebook for DTA metadata.

## Export Types
- **tsDTA**: Trial Specific DTA (Excel) - DTA Summary, Version History, Transfer Variables, Test Concepts
- **oa**: Operational Agreement (PDF) - Future
- **dta_full**: Complete package (ZIP) - Future

## Output Path
```
/Volumes/{catalog}/bronze_md/{source_root}/exports/{date}/{dta_number}/{filename}
```


In [ ]:
# Cell 1: Get Configuration from Setup Task
import json
from datetime import datetime

# Get globals from setup task
try:
    setup_result = dbutils.jobs.taskValues.get(taskKey="setup", key="globals")
    globals_dict = json.loads(setup_result) if isinstance(setup_result, str) else setup_result
    print(f"✓ Loaded globals from setup task")
except Exception as e:
    print(f"⚠ Could not load from setup task: {e}")
    globals_dict = {
        "catalog": "aira_test",
        "bronze_schema": "bronze_md",
        "silver_schema": "silver_md",
        "gold_schema": "gold_md"
    }

# Extract configuration
catalog = globals_dict.get("catalog", "aira_test")
bronze_schema = globals_dict.get("bronze_schema", "bronze_md")
silver_schema = globals_dict.get("silver_schema", "silver_md")
gold_schema = globals_dict.get("gold_schema", "gold_md")

print(f"Catalog: {catalog}")
print(f"Bronze Schema: {bronze_schema}")
print(f"Silver Schema: {silver_schema}")
print(f"Gold Schema: {gold_schema}")


In [ ]:
# Cell 2: Get Job Parameters

# Widget parameters (passed from job)
dbutils.widgets.text("dta_id", "", "DTA ID")
dbutils.widgets.text("export_type", "tsDTA", "Export Type")
dbutils.widgets.text("source_root", "clinical_data_standards/test", "Source Root")

dta_id = dbutils.widgets.get("dta_id")
export_type = dbutils.widgets.get("export_type")
source_root = dbutils.widgets.get("source_root")

# Validate required parameters
if not dta_id:
    raise ValueError("dta_id is required")

print(f"="*60)
print(f"EXPORT DTA FILE")
print(f"="*60)
print(f"DTA ID: {dta_id}")
print(f"Export Type: {export_type}")
print(f"Source Root: {source_root}")
print(f"="*60)


In [ ]:
# Cell 3: Load Export Configuration

# Try to load from config cache
try:
    config_df = spark.sql(f"""
        SELECT config_json 
        FROM {catalog}.{bronze_schema}.md_config_cache 
        WHERE config_key = 'clinical_data_standards' 
          AND config_section = 'export'
    """)
    
    if config_df.count() > 0:
        export_config = json.loads(config_df.first().config_json)
        print("✓ Loaded export config from cache")
    else:
        raise Exception("No config found")
except Exception as e:
    print(f"⚠ Using default export config: {e}")
    export_config = {
        "enabled": True,
        "export_folder": "exports",
        "date_format": "%Y-%m-%d",
        "timestamp_format": "%Y%m%d_%H%M%S",
        "document_types": [
            {
                "type": "tsDTA",
                "format": "excel",
                "file_extension": ".xlsx",
                "is_enabled": True,
                "includes": ["dta_summary", "version_history", "transfer_variables", "test_concepts"],
                "sheets_by_domain": True,
                "max_sheet_name_length": 31
            }
        ],
        "manifest": {"enabled": True, "filename": "export_manifest.json"}
    }

# Find the requested export type
export_type_config = None
for doc_type in export_config.get("document_types", []):
    if doc_type["type"] == export_type:
        export_type_config = doc_type
        break

if not export_type_config:
    raise ValueError(f"Export type '{export_type}' not found in configuration")

if not export_type_config.get("is_enabled", False):
    raise ValueError(f"Export type '{export_type}' is not enabled")

print(f"Export Type Config: {json.dumps(export_type_config, indent=2)}")


In [ ]:
# Cell 4: Get DTA Details

dta_table = f"{catalog}.{gold_schema}.dta"

dta_df = spark.sql(f"""
    SELECT *
    FROM {dta_table}
    WHERE dta_id = '{dta_id}'
""")

if dta_df.count() == 0:
    raise ValueError(f"DTA not found: {dta_id}")

dta = dta_df.first().asDict()
dta_number = dta.get("dta_number", "UNKNOWN")
dta_name = dta.get("dta_name", dta_number)

print(f"DTA: {dta_number} - {dta_name}")
print(f"Status: {dta.get('status')}")
print(f"Workflow: {dta.get('workflow_state')}")


In [ ]:
# Cell 5: Build Export Path
import os

date_format = export_config.get("date_format", "%Y-%m-%d")
timestamp_format = export_config.get("timestamp_format", "%Y%m%d_%H%M%S")
export_folder = export_config.get("export_folder", "exports")

date_str = datetime.now().strftime(date_format)
timestamp_str = datetime.now().strftime(timestamp_format)
file_extension = export_type_config.get("file_extension", ".xlsx")

# Build paths
export_dir = f"/Volumes/{catalog}/{bronze_schema}/{source_root}/{export_folder}/{date_str}/{dta_number}"
filename = f"{dta_number}_{export_type}_{timestamp_str}{file_extension}"
export_path = f"{export_dir}/{filename}"

print(f"Export Directory: {export_dir}")
print(f"Filename: {filename}")
print(f"Full Path: {export_path}")

# Create directory if it doesn't exist
dbutils.fs.mkdirs(export_dir.replace("/Volumes", "dbfs:/Volumes"))
print(f"✓ Created export directory")


In [ ]:
# Cell 6: Query Data for Export

export_data = {
    "dta": dta,
    "dta_number": dta_number,
    "export_timestamp": datetime.now().isoformat()
}

includes = export_type_config.get("includes", [])

# Version History
if "version_history" in includes:
    version_df = spark.sql(f"""
        SELECT 
            version, version_type, library_type, status,
            parent_version, record_count,
            created_by_principal, created_ts
        FROM {catalog}.{gold_schema}.md_version_registry
        WHERE dta_id = '{dta_id}'
        ORDER BY created_ts DESC
    """)
    export_data["version_history"] = version_df.toPandas()
    print(f"✓ Version History: {version_df.count()} records")

# Transfer Variables
if "transfer_variables" in includes:
    tv_df = spark.sql(f"""
        SELECT 
            transfer_variable_order, transfer_variable_name, transfer_variable_label,
            format, anticipated_max_length, transfer_file_key, populate_for_all_records,
            codelist_values, variable_description, status, domain_info, vendor_comment
        FROM {catalog}.{silver_schema}.md_dta_transfer_variables_draft
        WHERE dta_id = '{dta_id}'
        ORDER BY COALESCE(transfer_variable_order, 9999), transfer_variable_name
    """)
    export_data["transfer_variables"] = tv_df.toPandas()
    export_data["tv_domains"] = [row.domain_info for row in tv_df.select("domain_info").distinct().collect() if row.domain_info]
    print(f"✓ Transfer Variables: {tv_df.count()} records, {len(export_data['tv_domains'])} domains")

# Test Concepts
if "test_concepts" in includes:
    tc_df = spark.sql(f"""
        SELECT 
            test_concept_reference, transfer_tuple_map,
            status, domain_info, vendor_comment, notes
        FROM {catalog}.{silver_schema}.md_dta_vendor_test_concepts_draft
        WHERE dta_id = '{dta_id}'
        ORDER BY test_concept_reference
    """)
    export_data["test_concepts"] = tc_df.toPandas()
    export_data["tc_domains"] = [row.domain_info for row in tc_df.select("domain_info").distinct().collect() if row.domain_info]
    print(f"✓ Test Concepts: {tc_df.count()} records, {len(export_data['tc_domains'])} domains")


In [ ]:
# Cell 7: Generate Excel File - Setup and Helper Functions
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd

def style_header_row(ws, header_row=1):
    """Apply header styling to first row."""
    header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
    header_font = Font(color="FFFFFF", bold=True)
    thin_border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    
    for cell in ws[header_row]:
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = Alignment(horizontal='center', vertical='center')
        cell.border = thin_border

def add_dataframe_to_sheet(ws, df, start_row=1):
    """Add DataFrame to worksheet with headers."""
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), start_row):
        for c_idx, value in enumerate(row, 1):
            cell = ws.cell(row=r_idx, column=c_idx, value=value)
    style_header_row(ws, start_row)
    
    # Auto-adjust column widths
    for column in ws.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 50)
        ws.column_dimensions[column_letter].width = adjusted_width

def safe_sheet_name(name, max_length=31):
    """Create a safe Excel sheet name."""
    invalid_chars = [':', '\\', '/', '?', '*', '[', ']']
    for char in invalid_chars:
        name = name.replace(char, '_')
    return name[:max_length]

print("✓ Helper functions defined")


In [ ]:
# Cell 8: Create DTA Summary Sheet

print("Creating Excel workbook...")

wb = Workbook()
sheets_created = []

# Sheet 1: DTA Summary
ws_summary = wb.active
ws_summary.title = "DTA Summary"
sheets_created.append("DTA Summary")

# Add DTA details as key-value pairs
summary_data = [
    ["DTA Summary", ""],
    ["", ""],
    ["DTA ID", dta.get("dta_id", "")],
    ["DTA Number", dta.get("dta_number", "")],
    ["DTA Name", dta.get("dta_name", "")],
    ["Trial ID", dta.get("trial_id", "")],
    ["Data Stream", dta.get("data_stream_type", "")],
    ["Vendor", dta.get("data_provider_name", "")],
    ["", ""],
    ["Status", dta.get("status", "")],
    ["Workflow State", dta.get("workflow_state", "")],
    ["Current Version", dta.get("version", "")],
    ["Draft Version", dta.get("current_draft_version", "")],
    ["Base Template", dta.get("base_template_version", "")],
    ["", ""],
    ["Created By", dta.get("created_by_principal", "")],
    ["Created", str(dta.get("created_ts", ""))],
    ["Last Updated", str(dta.get("last_updated_ts", ""))],
    ["", ""],
    ["Notes", dta.get("notes", "")],
    ["", ""],
    ["Export Info", ""],
    ["Exported On", export_data["export_timestamp"]],
    ["Export Type", export_type]
]

for row_idx, row_data in enumerate(summary_data, 1):
    for col_idx, value in enumerate(row_data, 1):
        cell = ws_summary.cell(row=row_idx, column=col_idx, value=value)
        if col_idx == 1:
            cell.font = Font(bold=True)

ws_summary.column_dimensions['A'].width = 20
ws_summary.column_dimensions['B'].width = 50

print("✓ Created: DTA Summary")


In [ ]:
# Cell 9: Add Version History and Transfer Variables Sheets

# Version History Sheet
if "version_history" in export_data and len(export_data["version_history"]) > 0:
    ws_versions = wb.create_sheet("Version History")
    sheets_created.append("Version History")
    add_dataframe_to_sheet(ws_versions, export_data["version_history"])
    print(f"✓ Created: Version History ({len(export_data['version_history'])} records)")

# Transfer Variables Sheets (by domain)
sheets_by_domain = export_type_config.get("sheets_by_domain", True)
max_sheet_name_length = export_type_config.get("max_sheet_name_length", 31)

if "transfer_variables" in export_data and len(export_data["transfer_variables"]) > 0:
    tv_df = export_data["transfer_variables"]
    
    if sheets_by_domain and len(export_data.get("tv_domains", [])) > 0:
        # Create separate sheet per domain
        for domain in export_data["tv_domains"]:
            domain_df = tv_df[tv_df["domain_info"] == domain]
            if len(domain_df) > 0:
                sheet_name = safe_sheet_name(f"TV - {domain}", max_sheet_name_length)
                ws = wb.create_sheet(sheet_name)
                sheets_created.append(sheet_name)
                add_dataframe_to_sheet(ws, domain_df)
                print(f"✓ Created: {sheet_name} ({len(domain_df)} records)")
        
        # Add sheet for records without domain
        no_domain_df = tv_df[tv_df["domain_info"].isna() | (tv_df["domain_info"] == "")]
        if len(no_domain_df) > 0:
            sheet_name = safe_sheet_name("TV - General", max_sheet_name_length)
            ws = wb.create_sheet(sheet_name)
            sheets_created.append(sheet_name)
            add_dataframe_to_sheet(ws, no_domain_df)
            print(f"✓ Created: {sheet_name} ({len(no_domain_df)} records)")
    else:
        # Single sheet for all transfer variables
        sheet_name = "Transfer Variables"
        ws = wb.create_sheet(sheet_name)
        sheets_created.append(sheet_name)
        add_dataframe_to_sheet(ws, tv_df)
        print(f"✓ Created: {sheet_name} ({len(tv_df)} records)")


In [ ]:
# Cell 10: Add Test Concepts Sheets (by domain)

if "test_concepts" in export_data and len(export_data["test_concepts"]) > 0:
    tc_df = export_data["test_concepts"].copy()
    
    # Expand transfer_tuple_map for display
    if "transfer_tuple_map" in tc_df.columns:
        # Get all unique keys from the maps
        all_keys = set()
        for idx, row in tc_df.iterrows():
            if isinstance(row["transfer_tuple_map"], dict):
                all_keys.update(row["transfer_tuple_map"].keys())
        
        # Add columns for each key
        for key in sorted(all_keys):
            tc_df[key] = tc_df["transfer_tuple_map"].apply(
                lambda x: x.get(key, "") if isinstance(x, dict) else ""
            )
        
        # Drop the original map column for export
        tc_df = tc_df.drop(columns=["transfer_tuple_map"])
    
    if sheets_by_domain and len(export_data.get("tc_domains", [])) > 0:
        # Create separate sheet per domain
        for domain in export_data["tc_domains"]:
            domain_df = tc_df[tc_df["domain_info"] == domain]
            if len(domain_df) > 0:
                sheet_name = safe_sheet_name(f"TC - {domain}", max_sheet_name_length)
                ws = wb.create_sheet(sheet_name)
                sheets_created.append(sheet_name)
                add_dataframe_to_sheet(ws, domain_df)
                print(f"✓ Created: {sheet_name} ({len(domain_df)} records)")
        
        # Add sheet for records without domain
        no_domain_df = tc_df[tc_df["domain_info"].isna() | (tc_df["domain_info"] == "")]
        if len(no_domain_df) > 0:
            sheet_name = safe_sheet_name("TC - General", max_sheet_name_length)
            ws = wb.create_sheet(sheet_name)
            sheets_created.append(sheet_name)
            add_dataframe_to_sheet(ws, no_domain_df)
            print(f"✓ Created: {sheet_name} ({len(no_domain_df)} records)")
    else:
        # Single sheet for all test concepts
        sheet_name = "Test Concepts"
        ws = wb.create_sheet(sheet_name)
        sheets_created.append(sheet_name)
        add_dataframe_to_sheet(ws, tc_df)
        print(f"✓ Created: {sheet_name} ({len(tc_df)} records)")


In [ ]:
# Cell 11: Save Excel File to Volumes

print(f"\nSaving Excel file to: {export_path}")

# Save to local temp first, then copy to Volumes
import tempfile
import shutil

with tempfile.NamedTemporaryFile(suffix=".xlsx", delete=False) as tmp:
    temp_path = tmp.name

wb.save(temp_path)
print(f"✓ Saved to temp: {temp_path}")

# Copy to Volumes
shutil.copy(temp_path, export_path)
print(f"✓ Copied to Volumes: {export_path}")

# Verify file exists
file_size = os.path.getsize(export_path)
print(f"✓ File size: {file_size:,} bytes")

# Cleanup temp file
os.remove(temp_path)


In [ ]:
# Cell 12: Create Export Manifest and Set Task Values

manifest_config = export_config.get("manifest", {})

if manifest_config.get("enabled", True):
    manifest_filename = manifest_config.get("filename", "export_manifest.json")
    manifest_path = f"{export_dir}/{manifest_filename}"
    
    manifest = {
        "dta_id": dta_id,
        "dta_number": dta_number,
        "dta_name": dta_name,
        "export_type": export_type,
        "export_timestamp": export_data["export_timestamp"],
        "files": [
            {
                "filename": filename,
                "format": export_type_config.get("format"),
                "size_bytes": file_size,
                "sheets": sheets_created
            }
        ],
        "record_counts": {
            "transfer_variables": len(export_data.get("transfer_variables", [])),
            "test_concepts": len(export_data.get("test_concepts", [])),
            "version_history": len(export_data.get("version_history", []))
        },
        "source_root": source_root,
        "catalog": catalog
    }
    
    with open(manifest_path, 'w') as f:
        json.dump(manifest, f, indent=2, default=str)
    
    print(f"✓ Created manifest: {manifest_path}")

# Set Task Values for Downstream
result = {
    "success": True,
    "dta_id": dta_id,
    "dta_number": dta_number,
    "export_type": export_type,
    "export_path": export_path,
    "export_dir": export_dir,
    "filename": filename,
    "file_size": file_size,
    "sheets_created": sheets_created,
    "export_timestamp": export_data["export_timestamp"]
}

dbutils.jobs.taskValues.set(key="export_result", value=json.dumps(result))

print("\n" + "="*60)
print("EXPORT COMPLETE")
print("="*60)
print(f"DTA: {dta_number}")
print(f"Type: {export_type}")
print(f"File: {filename}")
print(f"Path: {export_path}")
print(f"Size: {file_size:,} bytes")
print(f"Sheets: {len(sheets_created)}")
for sheet in sheets_created:
    print(f"  - {sheet}")
print("="*60)
